# Graphing & manipulating data w/ `RMySQL`


## Loading required packages

In [1]:
require(ggplot2)  
require(dplyr)    
require(tidyr)    
require(magrittr) 
require(xlsx)     
require(repr)     
require(stringr)  
require(viridis)  
require(RMySQL)

Loading required package: ggplot2
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: tidyr
Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: xlsx
Loading required package: rJava
Loading required package: xlsxjars
Loading required package: repr
Loading required package: stringr
Loading required package: viridis
Loading required package: RMySQL
Loading required package: DBI


## Reading data

In [2]:
# password and IP redacted in public version of this notebook
PASSWORD <- '****'
MY_HOST_IP <- 'abc.xy.efg.wu'
# connect to the DB
# WILL ONLY WORK IF ON VPN
gnpn.db <- dbConnect(MySQL(), 
                          user='curtf', 
                          password=PASSWORD, 
                          dbname='gene_factory_db_II',
                          host=MY_HOST_IP,
                          port=3306)

# SQL queries get passed in as character vectors (aka strings)
my_query <- 'SELECT * FROM gene_info WHERE gene_sequence IS NOT NULL'

# doing a query is easy
my_result_object <- dbSendQuery(gnpn.db, my_query)

# getting the results as an R data frame is also easy
my_results_df <- fetch(my_result_object, n = -1)

Warning message:
In .local(conn, statement, ...): Decimal MySQL column 2 imported as numericWarning message:
In .local(conn, statement, ...): Unsigned INTEGER in col 8 imported as numeric

These were all R objects -- there was no need to save the query as a static .csv file and then import to R, for instance.  

In [3]:
head(my_results_df)
names(my_results_df)

,gene_id,gene_name,gene_version,genebank_id,gene_sequence,gene_function,chrs,locus,organism_id,notes
1,1,BFT4_1,1,NA,ATGCCCTGTCACGTCTTGCAAGTTGCGGCAGGTACTTACGATCTTCTTCACGGCATCGCAACCACAACCCCTTGGTCTGCCAGTATGATTTTTTTATTACTTTTGACCGCGTTCTTTGGCTCATATGAGAGTTTGATCAGTGCCTCACCGCTGAAGTCAACTTGCCGCTGCTTCCCAGGCGATTCTTGCTACCCTGATACTACAGCATGGGACGCCTTAAATGCTACGGTTGGAGGGAAGCTCATTGCAACAGTACCTATCGCTGCAGTGTGTCACTTTGATCAATTTGTGGCCTATGACGAAGCAGCATGTGCCACCTTGAGGGACAACTGGTTCTATCCTGAGACACATCTGCCTTCTCCGTCTTCTGCCATGGCCTGGCTGTTCACAAATAACACCTGTAATCCGTTCCTTTCATCTACAACCCCATGCACTTTGGGCAATTATGTGAGTTACACTATCAACGCTACCACAAGCAACGATGTGAAAGAAGCCATCAATTTTGCCAATTCGAATAATATCCGTCTGGTCATCCGCGGTACTGGCCATGATTATAATGGTAAATCGACAGGAGCCGGTGCACTCTCTATCTGGACGCATTATATGAAATTCATATCTCTCAATGAGTCGTATGAGTCTCAGGCGTACACCGGAAAATCAGTAGTTCTCGGTGCAGGTATCTCATCTATCGAAGCTTACAGATTTGCTGACGCGAACAACGGCTTGATTGTGGGAGGGAATTGTCCCACCGTCTCCTTGGCAGGGGGCTACACTCAAGGAGGAGGGCATGGTCCATTTGCGACCAAGTTTGGTCTTGCAGTTGACCAGGTTTTGGAATGGGAAGTGGTGACGAGTGAAGGAGAGCTATTGACCGCAAGTCCCACAATCAATTCTGATCTCTATTGGGCACTCTCTGGTGGAGGCGGAGGGACCTACGGCGTTGTATTATCGGTAACGGTCAAGTTTCATCCACCAACAAGCATACTATCTTCAGCAACTTTGCAGTTTGCTCCTCCAGCCACAGGGAACGCCACAGCATACTGGAATACAGTCAAAGTGTTTCTAGGATCTCTGCCAAGTTTGGTAGATGGTGGCTTGCAGCTTGGATGGACTCTTACACCCGAAGTTTTCTTGATCTCCCCAGTCACGGGTTTTGACGTCGAGCAAGCCAAGATTGACGAACTTTTTGCGCCAACGATATCGTCCCTTGAAGCAGCCAATATTTCGTATGCTTACACTTCTTCGGTAGCCACCAGCTTTCTCTCATATTACGAGTCCGGTGGCTTCGGCGCCAATGTCTCGAATACAAACCTTGCTGGCCGCTTACTCCCTCGTTCAATTGTCCAAAACAGCACCGATAGCTTCATCTCCATTCTCCAAACAATCGTCAACAACAAGCTTATTTTTGCCGGCGTCACCTTGGATGTAAATACACCTGTAGTAGCCAATGCGCCAGCCATCTCCATAAATCCTTACTGGAGACAAACTCTTATGACCGGCGTCTTCGGCGCCTACCTTGATTATACCGACATCGAACAAGGCTTCCAAACCCAAGACTTCATGACTGATACCATCATGCCTGCCTTGGCCGCCTTGACGCCAAACGGCGCCGCGTATATCAACGAAGCCGATTTTCAACAGCCGGACTGGCAGAATGTGTTTTATGGTGAGAATTTCAATCGCCTGAGTGATATAAAAGCGAAGTATGATCCCCGGCAAACATTTTACGCGCTGGGGGCAGTGGGGAGCGAGAAGTGGACAGAAAGATCGGATGGACGGCTTTGCAGGGTTTAG,FAD_binding_domain_protein,NA,NA,10,NA
2,2,BFT4_2,1,NA,ATGTCACAGCTTACTAGAAATATCCCGAGCCTTCAACTAAAGAATGGCTCCGCAATTCCCTTGCTTGCTTACGGCACTGGCTCCGTTATAAGCAAAGCAGCGGATGACGGCGCCACTGACGTAAAAATCGTCGAACACGTCCAATCAGCCCTCAAACTTGGCTACACGCACATCGACTGTGCTGAAGCCTACAACAATGAGCGTGAGGTCGGCGTCGCGATCCAGCAGAGCCAAATAGCTCGCGAGAAGCTCTTCGTGACTACGAAGACACTGGGTTTCCAGCCCATTGCTCAGGCGCTGGATGCCAGTTTGCAGAAACTGCAGCTCAGTTATGTAGACCTCTACCTCATCCACATCCCTTGGCTCGAGCCCACGCGCTCTCTCCGTGACGTATGGGCCGAGATGGAAGCCGTCAAAGCTTCGGGAAAAGCAAAAGCCATCGGCGTCTGTAACTTCGCCATTTCACACTTGGAGACTATCATAGCCGGTGCAAGCGAGATCCCGGCGGTTAATCAAATCGAGTATCACGCTTATCTCCAACAATCAGAGCTATCCGCGTACTGCCAGAAAAACGGTATAAAGGTAGAAGCTTTCGCGTCACTAGCACCGTGGATTACCGAGGGCCCGGCGAAGGGAGTAGTGGCGGAGCTTGCACAGAAGTATGGAGTTGGAGAGACGGAAATTTGCTTGAGGTGGTGTGTGCAGAAGGGGGTGCCAGTTGTGACGACGAGTTTGAAGGAGGAGAGGGTAAAAGGTTATCTAAAGGTTTTTGATTTTAGCCTTGAGGACAAGGAAATTGAACAGCTGTCGAGTGTTGGGGCGAGCCATCATTTCAGGAAGTATTTTGCTGACAATTTCGCTGGGGAGGATAAAGCATAG,Aldo_keto_reductase_oxidoreductase,NA,NA,10,NA
3,3,BFT4_3,1,NA,ATGGCATTAATACCTGGGCACTATCTTATTGCTGCTGCTGTTAGTGGTATCGCAGCCCACTTATGTGCCTTCATACACGGAGATTGGCACTTGCATATTCCTCAGGTTATCTTCACTCATTGTGCTGGCTATGCGTTACTTTTACTCAGGCTTAGATATCAAGAATTGAGCATCATAGACTCGATCCAGGCATCGACTGTCTTATCTGCATGCTATCTCTTATTCTTGTTCACCAGCATCATCGTCTACCGCGTCTTCTTCCATCGATTGCGGAATTTCCCTGGGCCTCGTCTTGCAGCAGCTACCAAGCTCTGGAGTATATGGGAAACAAGAGATTCCAGAAACCATCTGTTGATGCAACGGCTATTTGAGCAATATGGGCCAATAGTTAGAACGGCTCCCAATGAGCTCACGGTCTTTCATCCTGAAGGCGTTGAACTCGTGAAAGGAGGAAAAAATGGAACAGATCAGTATAACGAGAAGGGTGTTTGGTATGATCTAATTCATCCCAAAACATCGGTTGTGTTCAACCGTAATCCTTCCGAGAATGCCGTTCGACGTCGTCCTTGGGATCGAGCTGTTTCACCCGCAAGTCTTAGAAGTTATGCGGCAAACATAGTAGCCCCTGCCGCGGAAGTTCTGGAATTCGTTTCCAATGCCCAAGGCGAACCCGTCAATATTAATGAACTTATGACCGGACTCATCTTTGACTTTATGAGTGTGATCGTCTTCGGTGAAGTCTCTAGCAAAACCGATGCAGAGGAGCAAATTAGTACGCTGCGTAGACTCAAAGGGGCACTCATTTTACTAGCACCATGCGGAGATAGTGTCTGGCCAGCTCTTTTTGCCTTCAACTTTCTTCAGTTTTTGAAACCGGTCAAAAGCTGGCTAGGTCTTGTAGAGTCATGCCGCGCACGTATGCTGAAACGAATGAACAAAAAAACAAGCACAGTCGATGTTTCAGGCTTTTTCCTTAAAGATTTCTACGATTCGGCTGGGGAAACTAGTCTTGAGAAGCGTGAGAATGCGCTCCTCGGAAACACTATCTCTGCACTGGTTGCTGGTAGTGACACCGCACGTGCTGCCATGA

[1] "gene_id"       "gene_name"     "gene_version"  "genebank_id"  
 [5] "gene_sequence" "gene_function" "chrs"          "locus"        
 [9] "organism_id"   "notes"

In [4]:
dbListTables(gnpn.db)

[1] "assembly_overview_status_results_info"
 [2] "cds_info"                             
 [3] "chromatography_info"                  
 [4] "cluster_attributes_status_info"       
 [5] "cluster_cell_assembly"                
 [6] "column_view_dbtable_info"             
 [7] "enzyme_selection_info"                
 [8] "expression_cassette_info"             
 [9] "fermentation_info"                    
[10] "gene_info"                            
[11] "host_cell_info"                       
[12] "ms_analysis_features_info"            
[13] "organism_info"                        
[14] "plasmid_construct_info"               
[15] "plasmid_vector_info"                  
[16] "promoter_info"                        
[17] "results_and_analysis_info"            
[18] "terminator_info"                      
[19] "url_info"                             
[20] "view_assembly_details"                
[21] "view_chromotography_summary"          
[22] "view_expression_cassette"             
[23] "view_fermentation_summary"            
[24] "view_host_cell_assembly"              
[25] "view_mass_spec_features_summary"      
[26] "view_plasmid_construct"               
[27] "web_view_assembly_details"            
[28] "web_view_cluster_summary"             
[29] "web_view_landing_page"                
[30] "web_view_landing_page_ii"             
[31] "web_view_mass_spec_results"           
[32] "web_view_show_cds_info"               
[33] "web_view_show_gene_info"

In [5]:
dbListFields(gnpn.db, 'cds_info')

[1] "cds_id"        "cds_name"      "cds_version"   "gene_id"      
[5] "cds_sequence"  "cds_notes"     "cds_source"    "cds_parent_id"